In [46]:
import random
import networkx as nx


class Player:
    def __init__(self, name, start_node):
        self.name = name
        self.current_node = start_node
        self.path = [start_node]  # Track the path taken
     
    def move(self, new_node):
        self.current_node = new_node
        self.path.append(new_node)

class Attacker(Player):
    def __init__(self, name, start_node):
        super().__init__(name, start_node)

    def choose_target(self, targets):
        # Simple heuristic: pick the target with the highest value
        target = max(targets, key=lambda t: t.value)
        return target

class Defender(Player):
    def __init__(self, name, start_node, capture_radius):
        super().__init__(name, start_node)
        self.capture_radius = capture_radius
     
    def is_attacker_caught(self, attacker, game):
        # Check if attacker is within capture radius
        path_length = nx.shortest_path_length(game.graph, source=self.current_node, target=attacker.current_node)
        return path_length <= self.capture_radius

class Target:
    def __init__(self, node, value):
        self.node = node
        self.value = value


def format_strategies(strategy_matrix):
    """
    Post-processes a strategy matrix to a 2D array where each row is a strategy
    and each column corresponds to a timestep. Supports multiple players, and squeezes
    any extra dimensions.
    
    Input:
    - strategy_matrix: 3D numpy array of shape (num_strategies, num_timesteps, num_players)
    
    Returns:
    - 2D numpy array where each row is a strategy and each column corresponds to a timestep.
      If there's only one player, it removes the extra dimension.
    """
    num_strategies, num_timesteps, num_players = strategy_matrix.shape
    
    # Squeeze out the extra dimension if there's only one attacker
    if num_players == 1:
        formatted_strategies = np.squeeze(strategy_matrix, axis=2)
    else:
        formatted_strategies = []
        for strategy in strategy_matrix:
            formatted_strategy = []
            for timestep in range(num_timesteps):
                attacker_positions = tuple(strategy[timestep][a] for a in range(num_players))
                formatted_strategy.append(attacker_positions)
            formatted_strategies.append(formatted_strategy)
        
        formatted_strategies = np.array(formatted_strategies)
    
    return formatted_strategies


class InterdictionProtocol:
    def __init__(self, graph, defense_time_threshold):
        """
        Initialize the interdiction protocol.

        :param graph: The graph on which the game is played.
        :param defense_time_threshold: Number of timesteps a defender must spend at a target to successfully defend it.
        """
        self.graph = graph
        self.defense_time_threshold = defense_time_threshold

    def moving_interdiction(self, attacker_positions, defender_positions, capture_radii):
        """
        Determine which moving attackers are interdicted by moving defenders.

        :param attacker_positions: List of current positions of all moving attackers.
        :param defender_positions: List of current positions of all moving defenders.
        :param capture_radii: List of capture radii for each moving defender.
        :return: A list of booleans indicating whether each moving attacker is interdicted.
        """
        # print(defender_positions)
        interdicted = [False] * len(attacker_positions)

        for attacker_idx, attacker_position in enumerate(attacker_positions):
            for defender_idx, defender_position in enumerate(defender_positions):
                # Calculate shortest path distance
                distance = nx.shortest_path_length(
                    self.graph, source=attacker_position, target=defender_position
                )
                # Check if within capture radius
                # print(defender_idx)
                # print(capture_radii)
                if distance <= capture_radii[defender_idx]:
                    interdicted[attacker_idx] = True
                    break  # Stop checking other defenders for this attacker

        return interdicted

    def stationary_interdiction(self, stationary_attacker_positions, defender_strategy, defense_time):
        """
        Determine which stationary attackers are interdicted by defenders.

        :param stationary_attacker_positions: List of target nodes chosen by stationary attackers.
        :param defender_strategy: List of defender positions over all timesteps (list of lists).
        :param defense_time: Number of timesteps a defender must spend at a target to interdict it.
        :return: A list of booleans indicating whether each stationary attacker is interdicted.
        """
        # Count the number of timesteps defenders spend at each target
        defended_targets = {}
        for defender_positions in defender_strategy:
            for defender_position in defender_positions:
                if defender_position not in defended_targets:
                    defended_targets[defender_position] = 0
                defended_targets[defender_position] += 1

        # Determine interdiction for each stationary attacker
        interdicted = []
        for attacker_target in stationary_attacker_positions:
            if defended_targets.get(attacker_target, 0) >= defense_time:
                interdicted.append(True)
            else:
                interdicted.append(False)

        return interdicted


In [50]:
import numpy as np
import networkx as nx
import itertools

class Game:
    def __init__(self, graph, num_timesteps,moving_attacker_start_nodes, moving_defender_start_nodes, stationary_defender_start_nodes, targets, interdiction_protocol, num_moving_attackers=1, num_stationary_attackers=0, num_moving_defenders=1, num_stationary_defenders=0, allow_wait=True):
        self.graph = graph
        self.interdiction_protocol = interdiction_protocol
        self.num_timesteps = num_timesteps
        self.moving_attacker_start_nodes = moving_attacker_start_nodes
        self.moving_defender_start_nodes = moving_defender_start_nodes
        self.stationary_defender_start_nodes = stationary_defender_start_nodes
        self.num_moving_attackers = num_moving_attackers
        self.num_stationary_attackers = num_stationary_attackers
        self.num_moving_defenders = num_moving_defenders
        self.num_stationary_defenders = num_stationary_defenders
        self.allow_wait = allow_wait
        self.targets = targets
         
        # Initialize attacker and defender units
        self.moving_attacker_units = [Attacker(f"Moving Attacker {i+1}", start_node) for i, start_node in enumerate(moving_attacker_start_nodes[:num_moving_attackers])]
        self.stationary_attacker_units = [Attacker(f"Stationary Attacker {i+1}", None) for i in range(num_stationary_attackers)]
        self.moving_defender_units = [Defender(f"Moving Defender {i+1}", start_node, capture_radius=1) for i, start_node in enumerate(moving_defender_start_nodes[:num_moving_defenders])]
        self.stationary_defender_units = [Defender(f"Stationary Defender {i+1}", start_node, capture_radius=1) for i, start_node in enumerate(stationary_defender_start_nodes[:num_stationary_defenders])]

    # def generate_moving_player_strategies(self, graph, num_moving_units, start_nodes, num_timesteps, allow_wait):
    #     """
    #     Generates all possible movement strategies for moving players on the graph.
        
    #     Parameters:
    #     - graph: NetworkX graph representing the game environment.
    #     - start_nodes: List of potential start nodes for the players.
    #     - num_timesteps: Number of timesteps in the game.
    #     - allow_wait: Whether waiting at a node is allowed.
        
    #     Returns:
    #     - List of all possible paths (strategies) for the moving players.
    #     """
    #     all_paths = []
        
    #     # DFS logic to generate all possible strategies
    #     def dfs(current_positions, path):
    #         if len(path) == num_timesteps:
    #             all_paths.append(path)
    #             return
            
    #         for i, current_position in enumerate(current_positions):
    #             neighbors = list(graph.neighbors(current_position))
                
    #             # Add waiting option if allowed or no neighbors
    #             if allow_wait or not neighbors:
    #                 dfs(tuple(current_positions), path + [tuple(current_positions)])
                
    #             # Move to neighboring nodes
    #             for neighbor in neighbors:
    #                 new_positions = list(current_positions)
    #                 new_positions[i] = neighbor
    #                 dfs(tuple(new_positions), path + [tuple(new_positions)])
        
    #     # Initialize DFS for all combinations of start nodes
    #     for start_node_combination in itertools.product(start_nodes, repeat=num_moving_units):
    #         dfs(start_node_combination, [start_node_combination])
        
    #     return all_paths

    # def generate_strategy_matrix(self, player_type):
    #     """
    #     Generates a strategy matrix for the given player (Attacker or Defender) with support for
    #     both moving and stationary units.
        
    #     Parameters:
    #     - player_type: "attacker" or "defender"
        
    #     Returns:
    #     - strategy_matrix: 2D numpy array representing all possible strategies.
    #     """
    #     num_timesteps = self.num_timesteps
    #     allow_wait = self.allow_wait
        
    #     if player_type == "attacker":
    #         num_stationary_units = self.num_stationary_attackers
    #         num_moving_units = self.num_moving_attackers
    #         moving_start_nodes = self.moving_attacker_start_nodes
    #     elif player_type == "defender":
    #         num_stationary_units = self.num_stationary_defenders
    #         num_moving_units = self.num_moving_defenders
    #         moving_start_nodes = self.moving_defender_start_nodes
    #     else:
    #         raise ValueError("Invalid player_type. Choose 'attacker' or 'defender'.")
        
    #     # Generate strategies for moving units
    #     moving_paths = self.generate_moving_player_strategies(self.graph, num_moving_units, moving_start_nodes, num_timesteps, allow_wait)
        
    #     # Generate strategies for stationary units
    #     if player_type == "attacker":
    #         stationary_targets = [target.node for target in self.targets] + [None]
    #         stationary_strategies = list(itertools.product(stationary_targets, repeat=self.num_stationary_attackers))
    #     else:
    #         stationary_strategies = list(itertools.product(self.stationary_defender_start_nodes, repeat=self.num_stationary_defenders))
        
    #     # Combine moving and stationary strategies
    #     # print(moving_paths)
    #     # if num_stationary_units:
    #     #     combined_strategies = []
    #     #     for moving_path in moving_paths:
    #     #         for stationary_strategy in stationary_strategies:
    #     #             combined_path = [(move_pos, stationary_pos) for move_pos, stationary_pos in zip(moving_path,  [stationary_strategy]*num_timesteps)]
    #     #             combined_strategies.append(combined_path)
    #     # else:
    #     #     combined_strategies = moving_paths  # No stationary units

    #     if num_stationary_units:
    #         combined_strategies = []
    #         for moving_path in moving_paths:  # Each moving_path is a list of positions for moving units at each timestep
    #             for stationary_strategy in stationary_strategies:  # Each stationary_strategy is a tuple of stationary unit positions
    #                 # Combine moving and stationary positions for each timestep
    #                 combined_path = [
    #                     tuple(moving_positions) + stationary_strategy
    #                     for moving_positions in moving_path
    #                 ]
    #                 combined_strategies.append(combined_path)
    #     else:
    #         combined_strategies = moving_paths  # No stationary units
        
    #     # Convert to numpy array for consistency
    #     return np.array(combined_strategies)
    
    # def play_game_with_strategies(self, defender_strategy, attacker_strategy):
    #     """
    #     Play the game using specific strategies for both the defenders and attackers.
    #     The strategies are lists of nodes for each unit (both attackers and defenders).
    #     """
    #     # Initialize scores for each attacker
    #     reached_targets = set()  # This will store target nodes that have been reached
    #     attacker_scores = [0] * self.num_attackers
    #     interdicted = [False] * self.num_attackers  # Track which attackers have been interdicted
        
    #     # Loop over each timestep
    #     for t in range(self.num_timesteps):
    #         current_defender_positions = defender_strategy[t]  # Tuple of defender positions (moving, stationary)
    #         current_attacker_positions = attacker_strategy[t]  # Tuple of attacker positions
            
    #         # Ensure the node labels are correctly formatted as integers
    #         current_defender_positions = [
    #             int(d[0]) if isinstance(d, (list, np.ndarray)) else int(d) 
    #             for d in current_defender_positions
    #         ]
    #         current_attacker_positions = [
    #             int(a[0]) if isinstance(a, (list, np.ndarray)) else int(a) 
    #             for a in current_attacker_positions
    #         ]
            
    #         # Check for interdictions
    #         for attacker_idx, attacker_position in enumerate(current_attacker_positions):
    #             if interdicted[attacker_idx]:
    #                 continue  # Skip interdicted attackers
                
    #             # Compare each attacker position with all moving defender positions
    #             for defender_idx, defender_position in enumerate(current_defender_positions[:self.num_moving_defenders]):
    #                 if nx.shortest_path_length(self.graph, source=attacker_position, target=defender_position) <= self.moving_defender_units[defender_idx].capture_radius:
    #                     interdicted[attacker_idx] = True
    #                     break  # Interdiction occurred for this attacker
                
    #             # Compare attacker position with all stationary defender positions
    #             for defender_idx, defender_position in enumerate(current_defender_positions[self.num_moving_defenders:], start=0):
    #                 if nx.shortest_path_length(self.graph, source=attacker_position, target=defender_position) <= self.stationary_defender_units[defender_idx].capture_radius:
    #                     interdicted[attacker_idx] = True
    #                     break  # Interdiction occurred for this attacker
            
    #             # Update score for this attacker (if not interdicted yet)
    #             if not interdicted[attacker_idx]:
    #                 for target in self.targets:
    #                     if attacker_position == target.node and target.node not in reached_targets:
    #                         attacker_scores[attacker_idx] += -target.value
    #                         reached_targets.add(target.node)
        
    #         # If all attackers are interdicted, end the game early
    #         if all(interdicted):
    #             break
        
    #     # Return the total score (sum of all attackers' scores)
    #     return sum(attacker_scores)

    def generate_moving_player_strategies(self, graph, num_moving_units, start_nodes, num_timesteps, allow_wait):
        """
        Generates all possible movement strategies for moving players on the graph.
    
        Parameters:
        - graph: NetworkX graph representing the game environment.
        - start_nodes: List of potential start nodes for the players.
        - num_timesteps: Number of timesteps in the game.
        - allow_wait: Whether waiting at a node is allowed.
    
        Returns:
        - List of all possible paths (strategies) for the moving players.
        """
        if num_moving_units == 0:
            # No moving units, return an empty list
            return []
    
        all_paths = []
    
        # DFS logic to generate all possible strategies
        def dfs(current_positions, path):
            if len(path) == num_timesteps:
                all_paths.append(path)
                return
    
            for i, current_position in enumerate(current_positions):
                neighbors = list(graph.neighbors(current_position))
    
                # Add waiting option if allowed or no neighbors
                if allow_wait or not neighbors:
                    dfs(tuple(current_positions), path + [tuple(current_positions)])
    
                # Move to neighboring nodes
                for neighbor in neighbors:
                    new_positions = list(current_positions)
                    new_positions[i] = neighbor
                    dfs(tuple(new_positions), path + [tuple(new_positions)])
    
        # Initialize DFS for all combinations of start nodes
        for start_node_combination in itertools.product(start_nodes, repeat=num_moving_units):
            dfs(start_node_combination, [start_node_combination])
    
        return all_paths
    
    def generate_strategy_matrix(self, player_type):
        """
        Generates a strategy matrix for the given player (Attacker or Defender) with support for
        both moving and stationary units.
    
        Parameters:
        - player_type: "attacker" or "defender"
    
        Returns:
        - strategy_matrix: 2D numpy array representing all possible strategies.
        """
        num_timesteps = self.num_timesteps
        allow_wait = self.allow_wait
    
        if player_type == "attacker":
            num_stationary_units = self.num_stationary_attackers
            num_moving_units = self.num_moving_attackers
            moving_start_nodes = self.moving_attacker_start_nodes
        elif player_type == "defender":
            num_stationary_units = self.num_stationary_defenders
            num_moving_units = self.num_moving_defenders
            moving_start_nodes = self.moving_defender_start_nodes
        else:
            raise ValueError("Invalid player_type. Choose 'attacker' or 'defender'.")
    
        # Generate strategies for moving units
        moving_paths = self.generate_moving_player_strategies(
            self.graph, num_moving_units, moving_start_nodes, num_timesteps, allow_wait
        )
    
        # Generate strategies for stationary units
        if player_type == "attacker":
            stationary_targets = [target.node for target in self.targets] + [None]
            stationary_strategies = list(
                itertools.product(stationary_targets, repeat=num_stationary_units)
            )
        else:
            stationary_strategies = list(
                itertools.product(self.stationary_defender_start_nodes, repeat=num_stationary_units)
            )
    
        # Handle the case where there are no moving units
        if num_moving_units == 0:
            # Only stationary strategies
            combined_strategies = [
                [stationary_strategy] * num_timesteps
                for stationary_strategy in stationary_strategies
            ]
        elif num_stationary_units == 0:
            # Only moving strategies
            combined_strategies = moving_paths
        else:
            # Combine moving and stationary strategies
            combined_strategies = []
            for moving_path in moving_paths:  # Each moving_path is a list of positions for moving units at each timestep
                for stationary_strategy in stationary_strategies:  # Each stationary_strategy is a tuple of stationary unit positions
                    # Combine moving and stationary positions for each timestep
                    combined_path = [
                        tuple(moving_positions) + stationary_strategy
                        for moving_positions in moving_path
                    ]
                    combined_strategies.append(combined_path)
    
        # Convert to numpy array for consistency
        return np.array(combined_strategies)

    # def play_game_with_strategies(self, defender_strategy, attacker_strategy):
    #     """
    #     Play the game using specific strategies for both the defenders and attackers.
    #     The strategies are lists of nodes for each unit (both attackers and defenders).
    #     """
    #     reached_targets = set()  # Tracks which targets have been reached
    #     num_attackers = self.num_moving_attackers + self.num_stationary_attackers
    #     attacker_scores = [0] * num_attackers
    #     interdicted = [False] * num_attackers
    
    #     # Loop over each timestep
    #     for t in range(self.num_timesteps):
    #         current_defender_positions = defender_strategy[t]  # Tuple of defender positions
    #         current_attacker_positions = attacker_strategy[t]  # Tuple of attacker positions
    
    #         # Ensure node labels are integers
    #         current_defender_positions = [
    #             int(d[0]) if isinstance(d, (list, np.ndarray)) else int(d)
    #             for d in current_defender_positions
    #         ]
    #         current_attacker_positions = [
    #             int(a[0]) if isinstance(a, (list, np.ndarray)) else int(a)
    #             for a in current_attacker_positions
    #         ]
    
    #         # Check interdictions for moving attackers
    #         for attacker_idx, attacker_position in enumerate(current_attacker_positions[:self.num_moving_attackers]):
    #             if interdicted[attacker_idx]:
    #                 continue  # Skip interdicted attackers
                
    #             for defender_idx, defender_position in enumerate(current_defender_positions[:self.num_moving_defenders]):
    #                 if self.interdiction_protocol.moving_interdiction(
    #                     attacker_position, 
    #                     defender_position, 
    #                     self.moving_defender_units[defender_idx].capture_radius
    #                 ):
    #                     interdicted[attacker_idx] = True
    #                     break
    
    #         # Update score for moving attackers (not interdicted yet)
    #         for attacker_idx, attacker_position in enumerate(current_attacker_positions[:self.num_moving_attackers]):
    #             if not interdicted[attacker_idx]:
    #                 for target in self.targets:
    #                     if attacker_position == target.node and target.node not in reached_targets:
    #                         attacker_scores[attacker_idx] += -target.value
    #                         reached_targets.add(target.node)
    
    #         # Check stationary attackers' contributions
    #         stationary_attacker_positions = current_attacker_positions[self.num_moving_attackers:]
    #         stationary_interdictions = self.interdiction_protocol.stationary_interdiction(
    #             stationary_attacker_positions,
    #             defender_strategy,
    #             self.interdiction_protocol.defense_time_threshold
    #         )
    
    #         for idx, interdicted_stationary in enumerate(stationary_interdictions):
    #             if not interdicted_stationary:
    #                 for target in self.targets:
    #                     if stationary_attacker_positions[idx] == target.node and target.node not in reached_targets:
    #                         attacker_scores[self.num_moving_attackers + idx] += -target.value
    #                         reached_targets.add(target.node)
    
    #         # If all attackers are interdicted, end the game early
    #         if all(interdicted[:self.num_moving_attackers]) and all(stationary_interdictions):
    #             break
    
    #     # Return the total score (sum of all attackers' scores)
    #     return sum(attacker_scores)

    def play_game_with_strategies(self, defender_strategy, attacker_strategy):
        """
        Simulate the game with given defender and attacker strategies.
        """
        reached_targets = set()  # Tracks which targets have been reached
        moving_attacker_score = 0
        stationary_attacker_score = 0
    
        # Step 1: Handle moving attackers
        interdicted = [False] * self.num_moving_attackers
    
        for t in range(self.num_timesteps):
            # Get current positions for moving attackers and defenders
            current_attacker_positions = attacker_strategy[t][:self.num_moving_attackers]
            current_defender_positions = defender_strategy[t]
    
            # Use ip.moving_interdiction to determine which attackers are interdicted
            newly_interdicted = self.interdiction_protocol.moving_interdiction(
                current_attacker_positions, 
                current_defender_positions, 
                [unit.capture_radius for unit in self.moving_defender_units + self.stationary_defender_units]
            )
            
            # Update interdicted status
            for idx, is_interdicted in enumerate(newly_interdicted):
                interdicted[idx] = interdicted[idx] or is_interdicted
    
            # Skip processing for interdicted attackers
            for idx, position in enumerate(current_attacker_positions):
                if interdicted[idx]:
                    continue
    
                # Check if the attacker reached an unreached target
                for target in self.targets:
                    if position == target.node and target.node not in reached_targets:
                        moving_attacker_score -= target.value
                        reached_targets.add(target.node)
    
            # Exit early if all moving attackers are interdicted
            if all(interdicted):
                break
    
        # Step 2: Handle stationary attackers
        if self.num_stationary_attackers > 0:
            stationary_attacker_positions = [
                attacker_strategy[0][self.num_moving_attackers + idx]
                for idx in range(self.num_stationary_attackers)
            ]
    
            # Use ip.stationary_interdiction to determine stationary attacker interdictions
            stationary_interdictions = self.interdiction_protocol.stationary_interdiction(
                stationary_attacker_positions,
                defender_strategy,
                self.interdiction_protocol.defense_time_threshold
            )
    
            # Update scores for stationary attackers
            for idx, position in enumerate(stationary_attacker_positions):
                if not stationary_interdictions[idx] and position not in reached_targets:
                    for target in self.targets:
                        if position == target.node:
                            stationary_attacker_score -= target.value
                            reached_targets.add(target.node)
    
        # Step 3: Return the total score
        return moving_attacker_score + stationary_attacker_score



    # def generate_utility_matrix(self):
    #     """
    #     Generate the utility matrix by simulating all combinations of strategies
    #     for both attackers and defenders with multiple units.
    #     """
    
    #     defender_matrix = self.generate_strategy_matrix("defender")
    #     attacker_matrix = self.generate_strategy_matrix("attacker")
    
    #     utility_matrix = np.zeros((len(defender_matrix), len(attacker_matrix)))
    
    #     for i, defender_strategy in enumerate(defender_matrix):
    #         for j, attacker_strategy in enumerate(attacker_matrix):
    #             utility_matrix[i, j] = self.play_game_with_strategies(defender_strategy, attacker_strategy)
    
    #     return utility_matrix

    def generate_utility_matrix(self):
        """
        Generate the utility matrix by simulating all combinations of strategies
        for both attackers and defenders, accounting for stationary and moving units.
        """
        # Generate strategies for defenders and attackers
        defender_matrix = self.generate_strategy_matrix("defender")
        attacker_matrix = self.generate_strategy_matrix("attacker")
        
        # Initialize the utility matrix
        utility_matrix = np.zeros((len(defender_matrix), len(attacker_matrix)))
        
        # Loop through all combinations of strategies
        for i, defender_strategy in enumerate(defender_matrix):
            for j, attacker_strategy in enumerate(attacker_matrix):
                # Simulate the game with the current strategies and store the outcome
                utility_matrix[i, j] = self.play_game_with_strategies(defender_strategy, attacker_strategy)
        
        return utility_matrix

In [51]:
import sys
np.set_printoptions(threshold=sys.maxsize)
G = nx.grid_2d_graph(3, 3)
G = nx.convert_node_labels_to_integers(G)
targets = [Target(4,5), Target(7,3)]
ip = InterdictionProtocol(graph=G, defense_time_threshold=2)
game = Game(G, interdiction_protocol=ip, num_timesteps=4,
                    moving_attacker_start_nodes=[0, 3],
                    moving_defender_start_nodes=[8, 5],
                    stationary_defender_start_nodes=[2, 6],
                    num_moving_attackers=1 , 
                    num_stationary_attackers=1,
                    num_moving_defenders=1, 
                    num_stationary_defenders=1,
                    allow_wait=True,
                    targets = targets
                    )
# targets = [Target((0,1),5), Target((1,1),3)]
# game = SecurityGame(G, num_timesteps=41
#                     attacker_start_nodes=[(2, 2), (1, 2)],
#                     moving_defender_start_nodes=[(0, 0), (1, 0)],
#                     stationary_defender_start_nodes=[(2, 0), (0, 2)],
#                     num_attackers=1, 
#                     num_moving_defenders=1, 
#                     num_stationary_defenders=1,
#                     allow_wait=True,
#                     )

# set(game.generate_moving_player_strategies(game.graph, [0, 3], game.num_timesteps, game.allow_wait))

attacker_strategies = game.generate_strategy_matrix("attacker")
defender_strategies = game.generate_strategy_matrix("defender")

# len(format_strategies(attacker_strategies))
len(attacker_strategies)

297

In [33]:
attacker_strategies

array([[[0, 4],
        [0, 4],
        [0, 4],
        [0, 4]],

       [[0, 7],
        [0, 7],
        [0, 7],
        [0, 7]],

       [[0, None],
        [0, None],
        [0, None],
        [0, None]],

       [[0, 4],
        [0, 4],
        [0, 4],
        [3, 4]],

       [[0, 7],
        [0, 7],
        [0, 7],
        [3, 7]],

       [[0, None],
        [0, None],
        [0, None],
        [3, None]],

       [[0, 4],
        [0, 4],
        [0, 4],
        [1, 4]],

       [[0, 7],
        [0, 7],
        [0, 7],
        [1, 7]],

       [[0, None],
        [0, None],
        [0, None],
        [1, None]],

       [[0, 4],
        [0, 4],
        [3, 4],
        [3, 4]],

       [[0, 7],
        [0, 7],
        [3, 7],
        [3, 7]],

       [[0, None],
        [0, None],
        [3, None],
        [3, None]],

       [[0, 4],
        [0, 4],
        [3, 4],
        [0, 4]],

       [[0, 7],
        [0, 7],
        [3, 7],
        [0, 7]],

       [[0, None],
        [

In [34]:
format_strategies(defender_strategies)

array([[[8, 2],
        [8, 2],
        [8, 2],
        [8, 2]],

       [[8, 6],
        [8, 6],
        [8, 6],
        [8, 6]],

       [[8, 2],
        [8, 2],
        [8, 2],
        [5, 2]],

       [[8, 6],
        [8, 6],
        [8, 6],
        [5, 6]],

       [[8, 2],
        [8, 2],
        [8, 2],
        [7, 2]],

       [[8, 6],
        [8, 6],
        [8, 6],
        [7, 6]],

       [[8, 2],
        [8, 2],
        [5, 2],
        [5, 2]],

       [[8, 6],
        [8, 6],
        [5, 6],
        [5, 6]],

       [[8, 2],
        [8, 2],
        [5, 2],
        [2, 2]],

       [[8, 6],
        [8, 6],
        [5, 6],
        [2, 6]],

       [[8, 2],
        [8, 2],
        [5, 2],
        [4, 2]],

       [[8, 6],
        [8, 6],
        [5, 6],
        [4, 6]],

       [[8, 2],
        [8, 2],
        [5, 2],
        [8, 2]],

       [[8, 6],
        [8, 6],
        [5, 6],
        [8, 6]],

       [[8, 2],
        [8, 2],
        [7, 2],
        [7, 2]],

       [[8

In [35]:
len(defender_strategies)

198

In [54]:
game.generate_utility_matrix()

array([[-5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5.,
        -3.,  0., -5., -3.,  0., -5., -8., -5., -5., -3.,  0., -5., -3.,
         0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0.,
        -5., -3.,  0., -5., -8., -5., -5., -3.,  0., -5., -3.,  0., -5.,
        -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -8.,
        -5., -5., -8., -5., -5., -8., -5., -5., -8., -5., -5., -8., -5.,
        -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5.,
        -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,
         0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0.,
        -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5.,
        -3.,  0., -5., -8., -5., -5., -3.,  0., -5., -3.,  0., -5., -3.,
         0., -5., -3.,  0., -5., -3.,  0., -5., -3.,  0., -5., -8., -5.,
        -5., -8., -5., -5., -8., -5., -5., -8., -5., -5., -8., -5., -5.,
        -3.,  0., -5., -3.,  0., -5., -3.,  0., -5.

# NEXT STEP: CHECK IF THESE UTILITIES MAKE SENSE

In [18]:
print(game.targets[0].node, game.targets[0].value)
print(game.targets[1].node, game.targets[1].value)


4 5
7 3
